## 인셉션 모듈

* 3~4개의 가지를 가지고 하나의 concatenate 레이어로 결합되는 형태의 네트워크

!["Inception"](image/inception.png)


### 1x1  Convolution
* 일반적으로 각 브랜치의 시작은 **1x1 Convolution Layer**로 시작
  * 1x1  Convolution은 사용자가 지정한 만큼의 필터 갯수로 폭을 줄여 사용되는 파라미터 갯수를 줄임. 이렇게 채널 수를 줄였다가 다시 늘리는 부분을 bottleneck 구조라고 함.
  * 1x1이기 때문에 공간 방향으로는 정보를 섞지 않고, 채널 방향의 정보들만 혼합된 특성을 계산. 채널 방향과 공간 방향의 특성 학습을 분리할 수 있는 방법

!["bottleneck"](image/bottleneck.png)

In [5]:
from tensorflow.keras.datasets import mnist, cifar100, cifar10
from tensorflow.keras.models import Sequential, load_model, Model
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, concatenate
import tensorflow.keras
from tensorflow.keras import activations

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape(x_train.shape[0], 32, 32, 3).astype('float32')/255.
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3).astype('float32')/255.

y_train = tensorflow.keras.utils.to_categorical(y_train, 10)
y_test = tensorflow.keras.utils.to_categorical(y_test, 10)

In [ ]:
inputs = Input(shape=(32, 32, 3))

branch_a = Conv2D(16, (1, 1), activation="relu", strides=2)(inputs)

branch_b = Conv2D(16, (1, 1), activation="relu")(inputs)
branch_b = Conv2D(16, (3, 3), padding="same", activation="relu", strides=2)(branch_b)

branch_c = Conv2D(16, (1, 1), activation="relu")(inputs)
branch_c = Conv2D(16, (5, 5), padding="same", strides=2, activation="relu")(branch_c)

branch_d = MaxPooling2D(pool_size=(3, 3), strides=2, padding="same")(inputs)
branch_d = Conv2D(16, (1, 1), activation="relu")(branch_d)

# concat하는 출력의 차원이 모두 같아야 함
concat = concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

flatten = Flatten()(concat)

dense_1 = Dense(512, activation="relu")(flatten)
outputs = Dense(10, activation="softmax")(dense_1)

model = Model(inputs= inputs, outputs= outputs)

model.summary()

In [ ]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))



model.evaluate(x_test, y_test)

## 잔차 연결 (Residual Connection)

* 순서대로 놓인 네트워크를 가로질러 하위 층의 출력을 상위 층의 **출력** 에 덧셈으로 연결

* 깊은 네트워크에서 흔히 발생하는 
  * 기울기 소실(Vanishing Gradient)
  * 표현 병목(Representational Bottleneck, 히든 레이어의 유닛 수 부족으로 정보 전달이 잘 되지 않는 문제)
  
  두가지 문제를 해결하는데 도움을 줌. 일반적으로 10층 이상의 네트워크에서 사용
  
!["skip connection"](image/residual_connection.png)

In [6]:
# 특성 맵의 크기가 같을때

inputs = Input(shape=(32, 32, 3))

main_1 = Conv2D(3, (3, 3), activation="relu", padding="same")(inputs)
main_2 = Conv2D(3, (3, 3), activation="relu", padding="same")(main_1)
main_3 = Conv2D(3, (3, 3), activation="relu", padding="same")(main_2)

added = layers.add([main_3, inputs])

flatten = Flatten()(added)

dense_1 = Dense(128, activation="relu")(flatten)
outputs = Dense(10, activation="softmax")(dense_1)

model = Model(inputs= inputs, outputs= outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 3)    84          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 32, 32, 3)    84          conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 32, 32, 3)    84          conv2d_4[0][0]                   
______________________________________________________________________________________________

In [7]:
# 특성 맵의 크기가 다를때 

inputs = Input(shape=(32, 32, 3))

main_1 = Conv2D(3, (3, 3), activation="relu", padding="same")(inputs)
main_2 = Conv2D(6, (3, 3), activation="relu", padding="same")(main_1)
main_3 = MaxPooling2D(pool_size=(2, 2))(main_2)

# 임의의 레이어를 활용하여(ex. 1x1 Conv) 차원을 맞춰줘야 함
residual = Conv2D(6, (1, 1))(main_3)

added = layers.add([main_3, residual])

flatten = Flatten()(added)

dense_1 = Dense(128, activation="relu")(flatten)
outputs = Dense(10, activation="softmax")(dense_1)

model = Model(inputs= inputs, outputs= outputs)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 32, 32, 3)    84          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 32, 32, 6)    168         conv2d_6[0][0]                   
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 16, 16, 6)    0           conv2d_7[0][0]                   
____________________________________________________________________________________________